In [13]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from translation.translation_dataloader import TranslationDataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from translation.models.baseline_transformer.model import Seq2SeqTransformer
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import nltk

In [14]:
def build_text_transform():
    def sequential_transforms(*transforms):
        def func(txt_input):
            for transform in transforms:
                txt_input = transform(txt_input)
            return txt_input
        return func

    def tensor_transform(token_ids):
        return torch.cat((torch.tensor([BOS_IDX]),
                          torch.tensor(token_ids),
                          torch.tensor([EOS_IDX])))

    text_transform = sequential_transforms(token_transform,
                                               vocab_transform,
                                               tensor_transform)
    return text_transform

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(build_text_transform()(src_sample.rstrip("\n")))
        tgt_batch.append(build_text_transform()(tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = build_text_transform()(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def bleu_evaluation(transformer, test_iter):
    score = 0
    for sample, target in test_iter:
        prediction = translate(transformer, sample).split()
        print(sample, "-", " ".join(prediction))
        target = target.split()
        if len(sample) < 3 or len(target) < 3:
            BLEUscore = nltk.translate.bleu_score.sentence_bleu([target], prediction, weights=[0.5, 0.5, 0, 0])
        else:
            BLEUscore = nltk.translate.bleu_score.sentence_bleu([target], prediction)
        score += BLEUscore
    return score / len(test_iter)


In [15]:
data = TranslationDataset(("E:/Python/Projects/PSL-Translator/translation/data_collection/data.txt",))

In [16]:
token_transform = get_tokenizer('spacy', language='pl_core_news_sm')

In [17]:
def yield_tokens(data_iter: Iterable) -> List[str]:
    for data_sample in data_iter:
        yield token_transform(data_sample[0])
        yield token_transform(data_sample[1])

In [18]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [19]:
vocab_transform = build_vocab_from_iterator(yield_tokens(data),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform.set_default_index(UNK_IDX)

In [20]:
torch.manual_seed(0)

VOCAB_SIZE = len(vocab_transform)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

train_iter, val_iter, test_iter = torch.utils.data.random_split(data, [0.9, 0.05, 0.05])
train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, VOCAB_SIZE, VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

E:\Python\Projects\pythonProject4\venv\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [21]:

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    
    
    

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [22]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

E:\Python\Projects\pythonProject4\venv\lib\site-packages\torch\nn\functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 6.573, Val loss: 5.920, Epoch time = 0.956s
Epoch: 2, Train loss: 5.828, Val loss: 5.700, Epoch time = 0.704s
Epoch: 3, Train loss: 5.549, Val loss: 5.505, Epoch time = 0.701s
Epoch: 4, Train loss: 5.242, Val loss: 5.260, Epoch time = 0.695s
Epoch: 5, Train loss: 4.904, Val loss: 5.058, Epoch time = 0.690s
Epoch: 6, Train loss: 4.634, Val loss: 4.921, Epoch time = 0.690s
Epoch: 7, Train loss: 4.359, Val loss: 4.803, Epoch time = 0.679s
Epoch: 8, Train loss: 4.148, Val loss: 4.650, Epoch time = 0.675s
Epoch: 9, Train loss: 3.927, Val loss: 4.577, Epoch time = 0.692s
Epoch: 10, Train loss: 3.743, Val loss: 4.500, Epoch time = 0.677s
Epoch: 11, Train loss: 3.572, Val loss: 4.393, Epoch time = 0.687s
Epoch: 12, Train loss: 3.393, Val loss: 4.317, Epoch time = 0.699s
Epoch: 13, Train loss: 3.235, Val loss: 4.264, Epoch time = 0.705s
Epoch: 14, Train loss: 3.084, Val loss: 4.230, Epoch time = 0.709s
Epoch: 15, Train loss: 2.953, Val loss: 4.236, Epoch time = 0.697s
Epoc

In [23]:
print("Your BLEU score is: " + str(bleu_evaluation(transformer, train_iter)))

Ty mnie towarzyszyć? - Towarzyszysz mi ?
Ja książka w poniedziałek czyta było. - Czytałem książkę w poniedziałek .
Kot okno wskoczyć - Kot wskoczył na okno
Ty miasto duże co widzieć? - Jakie duże miasto widziałeś ?
Wakacje nad jezioro lubisz - Lubisz wakacje nad jeziorem
Ty rower naprawiać - Naprawiasz rower
Ty Polska być kiedy? - Kiedy byłeś w Polsce ?
Pies kość gryźć. - Pies gryzie kość .
Domek letniskowy ładny - Ładny domek letniskowy
On książkę czytać już - On już przeczytał książkę
My dom sprzątać wczoraj - Posprzątaliśmy dom wczoraj
My nowe mieszkanie znaleźć - Znaleźliśmy nowe mieszkanie
Jutro weekend odpoczynek potrzebować - Potrzebuję odpoczynku w weekend
Ona ostatnia noc źle spać - Źle spała ostatniej nocy
Ja wakacje Hiszpania spędzać - Spędziłem wakacje w Hiszpanii
My razem na koncert iść - Idziemy razem na koncert
Ona park spacerować - Ona spaceruje po parku
Dlaczego ty nie uczyć się wczoraj? - Dlaczego się wczoraj nie uczyłeś ?
Ja kino być chcieć - Chcę iść do kina
Ja cię 